In [ ]:
# /usr/bin/env python3
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re
import jieba
from collections import Counter

In [ ]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')

In [ ]:
df_train.head()
# df_test.head()

In [ ]:
# 标签比例
df_train['flag'].hist()

In [ ]:
df_train['len_title'] = df_train['title'].apply(lambda x: 0 if type(x) is float else len(x))
df_train['len_content'] = df_train['content'].apply(lambda x: 0 if type(x) is float else len(x))
df_train[['len_title','len_content']].describe()

In [ ]:
# 标题长度
df_train['title'].apply(lambda x: len(x)).hist(bins=100)

In [ ]:
# 查看超过150个字的标题
df_title_more150 = df_train[df_train['title'].apply(lambda x: len(x)>150)]
for index, row in df_title_more150.iterrows():
    print(row['title'])
    print(row['flag'])
    print('\n')

In [ ]:
'''
预处理掉类似这种内容：
1. ';}Ie_+='';Kc_(sy_,Ie_);}}function ht_(){var Sm_=aJ_();for (sy_=0; sy_< wF_.length;aT_++){var nv_=oX_(wF_[aT_],'_');Ie_+='';}Ie_+='';Kc_(sy_,Ie_);}else{var wF_= oX_(nK_[sy_],',');var Ie_='';for (aT_=0; aT_< wF_.length;aT_++){Ie_+=Mg_(wF_[
2. 緈冨_吶庅羙 2017/02/04 18:43:53 发表在 19楼 
'''
pat_case1 = r'[\'a-zA-Z_+=();}{,0-9<\s\[\]\.]{50,}'
pat_case2 = r'发表在.{0,5}(楼|板凳)'
instr1 = 'sampson0219 2017/01/08 09:56:02 发表在 板凳 同等 但是如果价位太高出来就可以放弃了 非要选gl8 不如选新款乞丐 低配20 高配24是有吸引力的 这还是建立在给的配置都不错上 毕竟2.5比不过2.0t 新款乞丐配置也不差 \
真皮很好解决 如果2.5减配严重 价位还不便宜 不如等等新款优惠 或者考虑别的确实，一看价格二看配置，\
离谱了就没几个人选了。等吧我也是等了好久了，新款一出来还真给吓着了，2.5的估计好不了哪去。发表在 板凳'
instr = "3.13号提车回泰安后发现车内太朴素～想这么调弄点亮条套餐威霆内饰亮条都塑料中网也有点失落过段时间中网行李驾还有腰线亮条作业我导航4500米算贵需要址我跟家说那些动动60001万你们哪来底气～真弄明白～ ';}oR_+='';iq_(TY_,oR_);}}function Pn_(){var Kx_=BQ_();for (TY_=0; TY_< mS_.length;RY_++){var Kh_=yO_(mS_[RY_],'_');oR_+='';}oR_+='';iq_(TY_,oR_);}else{v"
# pat = re.compile(pat_case2, re.M|re.I)
# obj = pat.search(instr)
# obj
# print(obj)
def remove_str(instr, pat):
    patc = re.compile(pat, re.M|re.I)
    obj = patc.search(instr)
    if obj is None:
        return instr
#     print(obj.group(0))
    pos = instr.find(obj.group(0))
    outstr = instr.replace(obj.group(0), '')
    return outstr
    
def process_case1(instr):
    return remove_str(instr, pat_case1)

def process_case2(instr):
    pat = re.compile(pat_case2, re.M|re.I)
    obj = pat.search(instr)
    if obj is None:
        return instr
    return instr[instr.find(obj.group(0))+len(obj.group(0)):]

print(process_case2(instr1))



In [ ]:
def process_str(instr):
    o_str = instr
    o_str = process_case1(o_str)
    o_str = process_case2(o_str)
    return o_str

In [ ]:
process_str(instr)

In [ ]:
# df_train['title'] = df_train['title'].apply(lambda x: process_str(x))

In [ ]:
# 查看超过150个字的标题
df_title_more150 = df_train[df_train['title'].apply(lambda x: len(x)>150)]
for index, row in df_title_more150.iterrows():
    print(row['title'])
    print(row['flag'])
    print('\n')

In [ ]:
# 查看不超过5个字的标题
df_title_more150 = df_train[df_train['title'].apply(lambda x: len(x)<15)]
for index, row in df_title_more150.iterrows():
    print(row['title'])
    print(row['flag'])
    print('\n')

In [ ]:
# 内容长度
df_train['content'].apply(lambda x: 0 if type(x) is float else len(x)).hist(bins=100)

In [ ]:
# 查看超过1500个字的内容
df_title_more150 = df_train[df_train['content'].apply(lambda x: type(x) is not float and len(x)>1500)]
for index, row in df_title_more150.iterrows():
    print(row['flag'])
    print(row['title'])
    print(row['content'])
    print('\n')

In [ ]:
# 训练集：检查是否标题是内容的一部分
cnt_total = 0
cnt_repetition = 0
for index, row in df_train.iterrows():
    cnt_total += 1
    if type(row['title']) is float or type(row['content']) is float:
        continue
    if row['title'] not in row['content']:
        cnt_repetition += 1
        print(row['flag'])
        print(row['title'])
        print(row['content'])
        print('\n')
print('{}/{} = {:.3f}'.format(cnt_repetition, cnt_total, cnt_repetition/cnt_total))

In [ ]:
# 测试集：检查是否标题是内容的一部分
cnt_total = 0
cnt_repetition = 0
for index, row in df_test.iterrows():
    cnt_total += 1
    if type(row['title']) is float or type(row['content']) is float:
        continue
    if row['title'] not in row['content']:
        cnt_repetition += 1
#         print(row['flag'])
        print(row['title'])
        print(row['content'])
        print('\n')
print('{}/{} = {:.3f}'.format(cnt_repetition, cnt_total, cnt_repetition/cnt_total))

In [ ]:
# 对所有的1数据进行分词，对所有的0数据进行分词，比对结果
all_words1 = []
all_words0 = []
for index, row in df_train.iterrows():
    if type(row['title']) is float or type(row['content']) is float:
        continue
    if row['flag'] == 1:
        all_words1 += list(jieba.cut(row['title']))
        all_words1 += list(jieba.cut(row['content']))
    else:
        all_words0 += list(jieba.cut(row['title']))
        all_words0 += list(jieba.cut(row['content']))
all_words1_dict = Counter(all_words1)
all_words0_dict = Counter(all_words0)
all_words1_dict_filtered = {k:v for k,v in all_words1_dict.items() if len(k) > 1 and v > 12}
all_words0_dict_filtered = {k:v for k,v in all_words0_dict.items() if len(k) > 1 and v > 12}

# 查找那些词只存在于1中
words_only_in1 = {}
for key, val in all_words1_dict_filtered.items():
    if key not in all_words0_dict:
        words_only_in1[key] = val
        
# 查找那些词只存在于0中
words_only_in0 = {}
for key, val in all_words0_dict_filtered.items():
    if key not in all_words1_dict:
        words_only_in0[key] = val
        
print(words_only_in1)
print('\n\n')
print(words_only_in0)

In [ ]:
a = '据了解，别克GL8车型只有单侧电动中门，而改装的左右两侧都电动的，但中门使用比较频繁的东西，很多车主都会为爱车加装电动中门功能，因为有了电动中门的GL8使用起来具备这些好处： 　　别克GL8改装电动门好处1：解决完美主义者单侧电动门的痛苦  　　别克GL8自带的电动中门，但乃至顶配也只配了一个右侧自动侧滑门，左侧仍然手动，这对于强迫症的车友来说，这非常一件令人痛苦的事情，改装左侧电动门后，左右对称了，痛苦也就消失了。 　　别克GL8改装自动侧滑门好处2：方便人员乘座车  　　全家出门旅游，慢慢走向停在路边的GL8，你提着大包小包，老婆一手提着大包行李，一手牵着淘气的孩子，还要照顾行动不便的老人，如果此时还要手动来打开中门，那是多么的不便。安装了电动中门就能在没人帮助的情况下，让中门自动打开，让所有人一次性安全上下车。 　　别克GL8改装自动侧滑门好处3：让客户满意的商务接待  　　客户越来越多，生意越来越好，今天有位重要客户前来，老总亲自前往迎接。将GL8车内喷上客户最爱的香水、车里播放最流行的音乐、放一本最新的时尚杂志，这些就够了吗？难道要让客户自己动手并使劲拉开中门？    　　GL8的第一大优点就是“大”。5米多的车里只放3排座位，腿部空间大得让任何人毫无怨言。很多车主为了进一步提升驾乘体验，会改装木地板，一般首选缅甸柚木，一千年不腐，二季节稳定性好。 　　整体铺设效果图。经典红木色，墨线精美、油影柔美、形成千姿百态的纹理结构，大气、典雅。 '
ccc = Counter(list(jieba.cut(a)))

In [ ]:
ccc1 = {k:v for k,v in ccc.items() if len(k) > 1 and v > 2}
ccc1

In [ ]:
for k,v in ccc1.items():
    print(k)

In [ ]:
df_result = pd.read_csv('all_info_pred_14_2.csv')
df_result_out = df_result.copy()

In [ ]:
print(df_result_out.head())

In [ ]:
df_result = pd.merge(df_result, df_test)
print(df_result.head())

In [ ]:
cnt_total = 0
cnt_repetition = 0
suspect_dict1 = {}
for index, row in df_result.iterrows():
    if row['flag'] != 1:
        continue
    cnt_total += 1
    concat_str = str(row['title']) + '\t\t\t\t' + str(row['content'])
    rep_time = 0  # 重复次数
    rep_words = []  # 包含于words_only_in0中，重复词的个数
    for key, val in words_only_in0.items():
        if key in concat_str:
            rep_time += 1
            rep_words.append(key)
#             break
    if len(rep_words) > 0:
        rep_words = ';'.join(rep_words)
        suspect_dict1[row['id']] = rep_words
#         print('===> {}\t\t{}'.format(rep_words, concat_str))
#         print('\n\n')
    cnt_repetition += 1 if rep_time > 0 else 0
print('{}/{} = {:.3f}'.format(cnt_repetition, cnt_total, cnt_repetition/cnt_total))
for k,v in suspect_dict1.items():
    print('{}:\t{}'.format(k,v))
# remove 64
df_result_new = df_result_out.copy()
for key,val in suspect_dict1.items():
    if val == '64':
        continue
    index = df_result_new[df_result_new['id'] == key].index
    df_result_new.iloc[index,3] = 0  # 1对应第1列（从第0列开始），根据自己的数据，应该改为3
    print(df_result_new.loc[index])
    print('convert {} 1 to 0'.format(key))

In [ ]:
cnt_total = 0
cnt_repetition = 0
suspect_dict0 = {}
for index, row in df_result.iterrows():
    if row['flag'] != 0:
        continue
    cnt_total += 1
    concat_str = str(row['title']) + '\t\t\t\t' + str(row['content'])
    rep_time = 0
    rep_words = []
    for key, val in words_only_in1.items():
        if key in concat_str:
            rep_time += 1
            rep_words.append(key)
#             break
    if len(rep_words) > 0:
        rep_words = ';'.join(rep_words)
        suspect_dict0[row['id']] = rep_words
        print('===> {}\t\t{}'.format(rep_words, concat_str))
        print('\n\n')
    cnt_repetition += 1 if rep_time > 0 else 0
print('{}/{} = {:.3f}'.format(cnt_repetition, cnt_total, cnt_repetition/cnt_total))
print(suspect_dict0)
for key,val in suspect_dict0.items():
    if val == '64':
        continue
    index = df_result_new[df_result_new['id'] == key].index
    df_result_new.iloc[index,3] = 1

In [ ]:
df_result_new.to_csv('all_info_pred_14_2_correct.csv', index=False, encoding='utf-8')

In [ ]:
df_result_new[['id', 'flag']].to_csv('pred_14_2_correct.csv', index=False, encoding='utf-8')